# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.131356e+02     1.748250e+00
 * time: 0.8554720878601074
     1     1.018375e+01     1.049257e+00
 * time: 3.497441053390503
     2    -1.187204e+01     1.281728e+00
 * time: 4.240815162658691
     3    -3.419856e+01     1.040504e+00
 * time: 5.254616022109985
     4    -4.768153e+01     7.784920e-01
 * time: 6.254950046539307
     5    -5.706383e+01     2.058158e-01
 * time: 7.283754110336304
     6    -5.983771e+01     1.398712e-01
 * time: 7.966370105743408
     7    -6.089792e+01     5.465006e-02
 * time: 8.626597166061401
     8    -6.132970e+01     5.901843e-02
 * time: 9.319226026535034
     9    -6.160511e+01     4.186675e-02
 * time: 10.00453805923462
    10    -6.182960e+01     3.060784e-02
 * time: 10.68055510520935
    11    -6.200059e+01     2.012101e-02
 * time: 11.423896074295044
    12    -6.206938e+01     1.736578e-02
 * time: 12.091700077056885
    13    -6.212895e+01     1.404404e-02
 * time: 12.779893159866333
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557663
    AtomicNonlocal      14.8522635
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336819

    total               -62.261666462208
